# 素数判定あれこれ

In [1]:
import math
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import *
from tqdm import tqdm_notebook as tqdm
from decimal import *
getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

## 素朴な素数判定

In [3]:
def is_prime(num):
    for k in range(2, int(math.sqrt(num))):
        if num%k == 0:
            return False
            break
    else: return True

In [ ]:
N=20
nlist = [(i,is_prime(i)) for i in range(2,N)]
print(nlist)

## 素数の個数

In [ ]:
Num = 3000
checklist = [(k, is_prime(k)) for k in range(2, Num+1)]

plist = [n[0] for n in checklist if n[1]] #= list(filter(lambda x: x[1], checklist))
print("There are {} prime numbers up to {}".format(len(plist), Num))

## 素数定理

In [ ]:
bp_n = range(5,Num+1)
bp_p = [len(list(filter((lambda z: z < x), plist))) for x in bp_n]

In [ ]:
bp_y = [x/(log(x)-1.08) for x in bp_n]

In [ ]:
# Make the plot
plt.figure(figsize=(16,10))

ax = plt.subplot()
ax.grid()
ax.set(xlabel='number (n)', ylabel='# of primes up to n')

plt.plot(bp_n, bp_p, color="blue")
plt.plot(bp_n, bp_y, color='green')
plt.show()

# 素数判定の高速化

In [4]:
%%timeit
Num = 10**5
checklist = [(k, is_prime(k)) for k in range(2, Num+1)]

plist = [n[0] for n in checklist if n[1]] #= list(filter(lambda x: x[1], checklist))
print("There are {} prime numbers up to {}".format(len(plist), Num))

There are 9679 prime numbers up to 100000
There are 9679 prime numbers up to 100000
There are 9679 prime numbers up to 100000
There are 9679 prime numbers up to 100000
1 loop, best of 3: 178 ms per loop


In [5]:
Num = 10**4
%timeit nlist=[k for k in range(2,Num+1) if k%3 == 0]
%timeit num = 5; nlist = [num*k for k in range(1,int(Num/num)+1)]

1000 loops, best of 3: 496 µs per loop
10000 loops, best of 3: 91.5 µs per loop


In [6]:
def find_factor_01(num):
    mx = int(math.sqrt(num))    
    count = 0
    for k in range(2, mx+1):
        count += 1
        if num%k==0: print(k)
            
def find_factor_02(num):
    mx = int(math.sqrt(num))
    composites = []
    for k in range(2, mx+1):
        if k not in composites:
            if num%k==0: print(k)
            else: composites.extend([k*i for i in range(2, int(mx/k)+1)])
                
def find_factor_03(num):
    nlist = list(range(2, int(math.sqrt(num))+1))
    while nlist:
        mm = nlist.pop(0)
        res = num%mm
        if res == 0:
            print(mm)
            break
        else:
            nlist = [k for k in nlist if k%mm > 0]

In [7]:
%load_ext line_profiler

In [12]:
num = 2**31-1
%lprun -f find_factor_02 find_factor_02(num)

```
Timer unit: 1e-06 s

Total time: 16.8452 s
File: <ipython-input-6-02803966a958>
Function: find_factor_02 at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     8                                           def find_factor_02(num):
     9         1          8.0      8.0      0.0      mx = int(math.sqrt(num))
    10         1          2.0      2.0      0.0      composites = []
    11     46340      38539.0      0.8      0.2      for k in range(2, mx+1):
    12     46339   16744590.0    361.3     99.4          if k not in composites:
    13      4792       8903.0      1.9      0.1              if num%k==0: print(k)
    14      4792      53194.0     11.1      0.3              else: composites.extend([k*i for i in range(2, int(mx/k)+1)])
```

## メルセンヌ素数

### メルセンヌ素数を探す

まずは素数判定関数を少し変更

In [ ]:
def find_factor(num):
    #getcontext().prec = Decimal(num).adjusted() + 2
    for k in range(2, int(math.sqrt(num))+1):
#        if Decimal(num)%k == 0:
        if num%k == 0:
            return k
            break
    else: return 0
    
def is_prime(num):
    return find_factor(num)==0

素数リスト `plist` の作成

In [ ]:
Num = 60
plist = [n for n in tqdm(range(2,Num+1)) if is_prime(n)]
print(plist)

メルセンヌ素数を素朴に探す。

In [ ]:
for k in tqdm(plist):
    fac = find_factor(2**k-1)
    if fac == 0:
        print('2^{}-1 = {} is prime'.format(k, 2**k-1))
    else:
        print('2^{}-1 = {} factors into {}*{}'.format(k, 2**k-1, fac, int((2**k-1)/fac)))

$2^{61}-1$ の素数判定はずいぶん時間がかかる。

In [ ]:
def find_factor_w_tqdm(num):
    for k in tqdm(range(2, int(math.sqrt(num))+1)):
        if num%k == 0:
            return k
            break
    else: return 0

In [ ]:
result = find_factor_w_tqdm(2**61-1); print(result)

### フェルマーテスト

素数判定をもっと高速のものにしたい。まずはフェルマーテスト。

In [ ]:
def fermat(num):
    num = abs(num)
    if num == 2:
        return True
    elif num < 2 or num%2 == 0:
        return False
    else:
        return pow(2, num-1, num) == 1

In [ ]:
for k in tqdm(plist):
    if fermat(2**k-1): print('2^{}-1 = {} is prime'.format(k, 2**k-1))

上の出力のように、$2^{11}-1$ や $2^{23}-1$ が素数と判定されてしまった。

### ミラーラビン法

確率的判定法。

In [ ]:
import random
def millerrabin(n):
    if n%2 == 0 or n%3 == 0 or n%5 == 0:
        return False
    else:
        s, d = 0, n-1
        while d%2==0: s,d = s+1, int(d/2)
        k = 50
#        for j in tqdm(range(k)):
        while k > 0:
            k = k-1
            a = random.randint(1,n-1)
            t, y = d, pow(a,d,n)
            while t != n-1 and y != 1 and y != n-1:
                y = pow(y,2,n)
                t <<= 1
            if y != n-1 and t%2 == 0:
                return False
        return True

In [ ]:
for k in tqdm(plist):
    if millerrabin(2**k-1): print('2^{}-1 = {} is prime'.format(k, 2**k-1))

ずいぶん時間がかかってしまう。なぜ?

### リュカ-レーマー・テスト

#### 疑似コード(from Wikipedia)
```
入力: p:奇素数であるテスト対象の整数
出力: PRIME:素数の場合, COMPOSIT:合成数の場合
Lucas_Lehmer_Test(p):
    var s = 4
    var M = 2**p − 1
    for n in range(2, p):
        s = (s*s-2)%M
    if s == 0 then
        return PRIME
    else
        return COMPOSIT
```

In [ ]:
def lucas_lehmer(p):
    s = 4
    mp = (1<<p)-1
    for n in range(p-2): # p-2 times iteration (list(range(p-2))=[0,1,...,8])
         s = (s**2-2)%mp
    return s==0

In [ ]:
def lucas_lehmer_FAST(p):
    s = 4
    mp = (1<<p)-1
    for n in range(p-2):
        ss = s*s        
        s = (ss & mp) + (ss >> p)
        if s >= mp: s = s-mp
        s = s-2
    return s==0

In [ ]:
lucas_lehmer(4423)

In [ ]:
lucas_lehmer_FAST(4423)

In [ ]:
def mp_str(p):
    mp = Decimal(2**p-1)
    nod = mp.adjusted()+1
    if nod <= 40:
        return str(mp)
    else:
        getcontext().prec = nod
        top = (mp/(Decimal(10)**(nod-20))).quantize(1)
        bottom = mp-math.floor(mp/Decimal(1.0e+20))*Decimal(1.0e+20)
        return '{}...{} [{} digits]'.format(top,bottom,nod)

def find_mp(nmin, nmax):
    plist = [n for n in range(nmin, nmax+1) if is_prime(n)]
    for p in tqdm(plist):
        if lucas_lehmer_FAST(p): print('2^{}-1 = {} is prime'.format(p, mp_str(p)))

In [ ]:
find_mp(100, 1000)

In [ ]:
find_mp(1000, 4000)

In [ ]:
find_mp(10000, 20000)

In [ ]:
find_mp(20000, 50000)